훈련 데이터 전처리하기

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [3]:
vocab = list(set(sentence))
print(vocab)

['memory', 'medicine', 'Repeat', 'the', 'for', 'is', 'best']


In [4]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [5]:
print(word2index)

{'memory': 1, 'medicine': 2, 'Repeat': 3, 'the': 4, 'for': 5, 'is': 6, 'best': 7, '<unk>': 0}


In [6]:
print(word2index['memory'])

1


In [7]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'memory', 2: 'medicine', 3: 'Repeat', 4: 'the', 5: 'for', 6: 'is', 7: 'best', 0: '<unk>'}


In [8]:
print(index2word)


{1: 'memory', 2: 'medicine', 3: 'Repeat', 4: 'the', 5: 'for', 6: 'is', 7: 'best', 0: '<unk>'}


In [9]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [10]:
X, Y = build_data(sentence, word2index)

In [11]:
print(X)
print(Y)

tensor([[3, 6, 4, 7, 2, 5]])
tensor([[6, 4, 7, 2, 5, 1]])


모델 구현하기

In [12]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [13]:
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [14]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [15]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)

tensor([[-0.0722, -0.4866, -0.1651, -0.0027, -0.4492, -0.0185, -0.2332, -0.0024],
        [ 0.5919, -0.1584, -0.0534,  0.0319, -0.3154,  0.2142, -0.2951,  0.1597],
        [ 0.2348,  0.0126,  0.0412,  0.1015, -0.3246,  0.0879, -0.3190, -0.0191],
        [ 0.2270, -0.0353, -0.1465,  0.1594, -0.3288,  0.2464,  0.1469, -0.3500],
        [-0.1923, -0.7129, -0.3407,  0.0474, -0.5790, -0.1050, -0.2720, -0.1388],
        [ 0.3253,  0.1247,  0.4744, -0.0132, -0.0514,  0.3169, -0.2020,  0.2753]],
       grad_fn=<ViewBackward>)


In [16]:
print(output.shape)
torch.Size([6, 8])

torch.Size([6, 8])


torch.Size([6, 8])

In [17]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [18]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.1666 
Repeat best <unk> <unk> for Repeat medicine

[41/201] 1.5506 
Repeat for <unk> best medicine for memory

[81/201] 0.9149 
Repeat is the best medicine for memory

[121/201] 0.4875 
Repeat is the best medicine for memory

[161/201] 0.2651 
Repeat is the best medicine for memory

[201/201] 0.1561 
Repeat is the best medicine for memory

